<h3> Imports </h3>

In [1]:
import ee
import os
import numpy as np
import geemap
ee.Initialize()

In [2]:
IPIS = ee.FeatureCollection("users/rishiAgarwal/Congo_Active_Mines")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
visualization = {
    "min": 0,
    "max": 0.3,
    "bands": ["B4","B3","B2"]
}
visParams = {'color': 'blue'}
# cloud mask sentinel
def maskS2clouds(image):
    qa = image.select('QA60')

    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).select("B.*") \
        .copyProperties(image, ["system:time_start"])

<h3> Essential Functions </h3>

In [3]:
def convertToFeatureCollection(csv):
    points = np.genfromtxt(csv, delimiter=',', skip_header=1)
    feature_array = points
    region_list = []
    for row in feature_array:
        region = ee.Geometry.Polygon([[[float(row[0]), float(row[3])],
                                      [float(row[0]), float(row[1])],
                                      [float(row[2]), float(row[1])],
                                      [float(row[2]), float(row[3])]]])
        f = ee.Feature(region).set('elevation score', float(row[17])).set('band score', float(row[18]))
        region_list.append(f)

    # Wrap feature list in a Feature Collection
    fc = ee.FeatureCollection(region_list)
    return fc

In [4]:
def visArea(min_lon, max_lon, max_lat, min_lat):
    Map = geemap.Map()
    area = ee.Geometry.Rectangle(min_lon, min_lat, max_lon, max_lat)
    filtered = s2 \
    .filter(ee.Filter.bounds(area)) \
    .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
    .map(maskS2clouds) \
    .select('B.*')
    composite = filtered.median().clip(area)
    return composite

In [5]:
def nearestNeighbor(result_filepath, ipis_dataset):
    os.system('python3 nearest_neighbors.py ' + result_filepath + ' ' + ipis_dataset)

<h3> CSVs Filepaths </h3>

In [6]:
r_26_n7 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/26_-7_passing.csv"
r_26_n8 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/26_-8_passing.csv"
r_27_n7 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/27_-7_passing.csv"
r_27_n8 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/27_-8_passing.csv"
r_28_n7 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/28_-7_passing.csv"
r_28_n8 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/28_-8_passing.csv"
r_29_n7 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/29_-7_passing.csv"
r_29_n8 = "/scratch/agarwal.rishi/bigRegion_csvs/passing/29_-8_passing.csv"
big_region_2575_n15 = "/scratch/agarwal.rishi/GEE/rishiAgarwal/Jobs/final/Visualize/compiled_25.75_-1.5_passing.csv"
big_region_2575_n25 = "/scratch/agarwal.rishi/GEE/rishiAgarwal/Jobs/final/Visualize/compiled_25.75_-2.5_passing.csv"
r_highres_ray = "/scratch/agarwal.rishi/bigRegion_csvs/passing/highres_ray_passing.csv"

<h3> Ipis Dataset Filepaths </h3>

In [7]:
ipis_26_n7 = "Ipis_datasets/ipis_26_-7.csv"
ipis_26_n8 = "Ipis_datasets/ipis_26_-8.csv"
ipis_27_n7 = "Ipis_datasets/ipis_27_-7.csv"
ipis_27_n8 = "Ipis_datasets/ipis_27_-8.csv"
ipis_28_n7 = "Ipis_datasets/ipis_28_-7.csv"
ipis_28_n8 = "Ipis_datasets/ipis_28_-8.csv"
ipis_29_n7 = "Ipis_datasets/ipis_29_-7.csv"
ipis_29_n8 = "Ipis_datasets/ipis_29_-8.csv"

<h3> Visualizations </h3>

In [11]:
table = convertToFeatureCollection(big_region_2575_n15)
table_2 = convertToFeatureCollection(big_region_2575_n25)
composite = visArea(25.75, 26.75, -2.5, 0.5)
Map = geemap.Map()
Map.addLayer(composite, visualization, 'Composite Image')
Map.addLayer(IPIS, visParams, "mines", opacity=1)
Map.addLayer(table, {'color': 'red'}, "Passing regions red", opacity=1)
Map.addLayer(table_2, {'color': 'green'}, "Passing regions green", opacity=1)
Map.centerObject(composite, 9)
Map


Map(center=[-0.9998095084397712, 26.249999999999975], controls=(WidgetControl(options=['position', 'transparen…

<h3> Nearest Neighbor Information </h3>

In [13]:
nearestNeighbor(r_26_n8, ipis_26_n8)

% within 1km of a mapbox point: 0.0
% within 5km of a mapbox point: 0.0
% within 10km of a mapbox point: 10.81081081081081
